In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import os
import re
from css_utility import *

In [5]:
crm_path="../database/remap2022/remap2022_crm_macs2_hg19_v1_0.bed"
gene_path="../database/RefSeq/RefSeq.WholeGene.bed"
cell_sample_path="../database/roadmap/df_pickled/E003_df_pickled.pkl"
css_sample_path="../database/roadmap/css_unit_pickled/E003_unitcss_woChrM.pkl"


In [167]:
with open(cell_sample_path, "rb") as f:
    cell_sample=pickle.load(f)

In [39]:
gene_df=pd.read_csv(gene_path,sep='\t', header=None)
gene_df.rename(columns={0:"chromosome",1:"TxStart",2:"TxEnd",3:"name",4:"unk0",
                                  5:'strand', 6:'cdsStart', 7:'cdsEnd',8:"unk1",9:"exonCount",
                                  10:"unk2",11:"unk3"},inplace=True)
gene_df.head()

,chromosome,TxStart,TxEnd,name,unk0,strand,cdsStart,cdsEnd,unk1,exonCount,unk2,unk3
0,chr1,66999638,67216822,NM_032291,0,+,67000041,67208778,0,25,"413,64,25,72,57,55,176,12,12,25,52,86,93,75,501,128,127,60,112,156,133,203,65,165,8067,","0,91891,99114,101988,105821,108854,109588,126557,133574,137039,137988,139325,143048,145722,147913,155192,156234,161478,185338,195308,199792,205379,206702,207316,209117,"
1,chr1,66999251,67216822,NM_001308203,0,+,67000041,67208778,0,22,"104,123,64,25,57,55,176,25,52,86,93,75,128,127,66,112,156,133,203,65,165,8067,","0,677,92278,99501,106208,109241,109975,137426,138375,139712,143435,146109,155579,156621,160870,185725,195695,200179,205766,207089,207703,209504,"
2,chr1,33547778,33586132,NM_001301825,0,+,33547850,33585783,0,9,"177,174,173,135,166,163,173,215,488,","0,1776,9872,11104,12370,14529,15829,35724,37866,"
3,chr1,8378144,8404227,NM_001080397,0,+,8378168,8404073,0,9,"102,421,93,225,728,154,177,206,421,","0,6221,7213,7733,12124,17352,19731,21408,25662,"
4,chr1,16767166,16786584,NM_001145278,0,+,16767256,16785385,0,8,"104,101,105,82,109,178,76,1248,","0,2960,7198,7388,8421,11166,15146,18170,"


In [47]:
#### unit-css per chromosome of one cell
with open(css_sample_path, "rb") as s:
    test_unit_css=pickle.load(s)

In [79]:
len(test_unit_css)

24

In [6]:
import pandas as pd

def crm_df_maker(crm_path="../database/remap2022/remap2022_crm_macs2_hg19_v1_0.bed", limit_len=3):
    # Load the data
    crm_raw = pd.read_csv(crm_path, sep='\t', header=None, names=["chromosome", "start", "end", "name", "score", "strand", "thickStart", "thickEnd", "itemRgb"])
    
    # Convert start and end locations to units of 200 bps and calculate length
    crm_raw['start'] = (crm_raw['start'] / 200).round().astype(int)
    crm_raw['end'] = (crm_raw['end'] / 200).round().astype(int)
    crm_raw['length'] = crm_raw['end'] - crm_raw['start'] + 1

    # Filter by chromosome and length
    crm_df = crm_raw[crm_raw["chromosome"].str.contains('^chr[0-9XY]+$') & (crm_raw['length'] >= limit_len)].copy()
    
    # Define chromosome order
    chromosome_order = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10',
                        'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19',
                        'chr20', 'chr21', 'chr22', 'chrX', 'chrY']

    # Convert 'chromosome' to a categorical type with the defined order
    crm_df['chromosome'] = pd.Categorical(crm_df['chromosome'], categories=chromosome_order, ordered=True)

    # Sort by 'chromosome' and 'start'
    crm_df = crm_df.sort_values(['chromosome', 'start'])
    crm_df_fin = crm_df[["chromosome","start" ,"end","length","name"]]

    # Print summary
    print("{} out of total {} CRM entries are longer than 200x{}, which is approx. {} %".format(len(crm_df), len(crm_raw), limit_len, round(len(crm_df)/len(crm_raw), 3)))
    
    return crm_df_fin


In [38]:
crm_len4_df=crm_df_maker(crm_path="../database/remap2022/remap2022_crm_macs2_hg19_v1_0.bed",limit_len=4)

1113643 out of total 3448721 CRM entries are longer than 200x4, which is approx. 0.323 %


In [39]:
crm_len4_df.head(10)

,chromosome,start,end,length,name
2,chr1,682,686,5,"NCOR1,HDAC1,MTA3,L3MBTL2,DPF2,ZBTB33,HES1,HDGF..."
11,chr1,2825,2828,4,"SRSF7,PCBP1,RBM22,PCBP2,TAF15,AGO1,NANOG,SOX4,..."
13,chr1,2833,2836,4,"SMARCC1,E2F1,HSF1,TARDBP,STAT3,SRC,RELA,HNRNPH..."
18,chr1,3569,3572,4,"TOP1,CEBPB,CSNK2A1,KDM4C,PHF8,ZNF18,ZBTB2,FOSL..."
23,chr1,3625,3628,4,"ZNF530,HSF1,DPF2,IKZF2,IKZF1"
27,chr1,3762,3765,4,"CREBBP,SOX2,ILF3,NFIC,CTCF,NR3C1,BRD4,HNF4A,ME..."
29,chr1,3776,3780,5,"KLF4,ERG,ZSCAN29,ZEB2,ZEB1,ZNF189,HIC1,HDGF,ZN..."
31,chr1,3783,3786,4,"BCL11A,CLOCK,ZNF143,ZNF143"
32,chr1,3786,3790,5,"ZNF189,ZEB1,ZNF580,HDAC2,ZEB2,PRDM2,HIC1,CTCF,..."
35,chr1,3793,3796,4,"ZNF143,ILF3,ZNF580,ZEB1,ZNF189,ERG,ZNF24,SMAD2-3"


In [7]:
### cut the css according to the CRM position
def extCRMfromCell(css_sample_path="../database/roadmap/css_unit_pickled/E003_unitcss_woChrM.pkl",crm_path="../database/remap2022/remap2022_crm_macs2_hg19_v1_0.bed",limit_len=4):
    #### load unit-css per chromosome of one cell
    with open(css_sample_path, "rb") as s:
        unit_css=pickle.load(s)
    #### make CRM as a dataframe
    crm_df_fin=crm_df_maker(crm_path=crm_path,limit_len=limit_len)
    
    cut_lst_all=[]
    for chr in range(len(unit_css)):
        unit_css_chr=unit_css[chr]
        chromosome_order = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10',
                            'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19',
                            'chr20', 'chr21', 'chr22', 'chrX', 'chrY']
        crm_df_chr=crm_df_fin[crm_df_fin["chromosome"]==chromosome_order[chr]]

        cut_lst=[]
        for i in range(len(crm_df_chr)):
            start_loc=crm_df_chr["start"].iloc[i]
            end_loc=crm_df_chr["end"].iloc[i]
            # length=crm_df_fin["length"].iloc[i]     
            cut=unit_css_chr[start_loc:end_loc+1]
            cut_lst.append(cut)
        cut_lst_all.append(cut_lst)
    return cut_lst_all


In [41]:
cut_lst_all=extCRMfromCell(css_sample_path="../database/roadmap/css_unit_pickled/E003_unitcss_woChrM.pkl",crm_path="../database/remap2022/remap2022_crm_macs2_hg19_v1_0.bed",limit_len=6)

97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %


In [42]:
cut_lst_all[0]

['MMMMMM',
 'LLLLLMM',
 'MMMMML',
 'DDDDDD',
 'BBBGNN',
 'NNNNNNNN',
 'MMMMMMM',
 'MMLLLL',
 'KKJKJJ',
 'JJJMMMM',
 'LLLLLL',
 'LMMMMMNN',
 'NGBAABB',
 'AABBGG',
 'GFFFFF',
 'GGGGGG',
 'MMMMMM',
 'MMMMLLK',
 'LLLNNN',
 'OOOOOO',
 'AAAAAB',
 'MMMMMN',
 'GGGGOOO',
 'MNNNNN',
 'BBGGGGEE',
 'KLLLLG',
 'EEEEEE',
 'NNLLGG',
 'NNNNNNN',
 'NNNNNNN',
 'DDDDDD',
 'DDDDDD',
 'DDDDDD',
 'EEEEEEE',
 'CFFFEE',
 'EEGGGE',
 'EEEEEE',
 'NNNNLLL',
 'MMMMME',
 'EEEEEE',
 'EEDDDD',
 'DDDDDDD',
 'EEEEEEE',
 'EEEBBB',
 'NNNNNN',
 'EEEEEE',
 'EEEEEEE',
 'EEEDDDDE',
 'EEEEEE',
 'EEEEEE',
 'EGGGGG',
 'GGEEEE',
 'AAAAEE',
 'DDDDDD',
 'FFFDDD',
 'GGGGGG',
 'GGGGGG',
 'BBBAAA',
 'AAAAAA',
 'AAAAEE',
 'DDDDDD',
 'EEEEEE',
 'DDDDDD',
 'NNNNNN',
 'NNNNNNN',
 'LLLLLL',
 'LKKJJJJ',
 'JJJKLM',
 'OOOOOO',
 'JJJJJE',
 'DDDDDD',
 'EEEGGG',
 'GGGGEE',
 'DDDDDD',
 'DDDDDD',
 'AABBBB',
 'BAAGGGG',
 'GGGBBB',
 'DDDDDD',
 'DDDDDD',
 'DDDEEE',
 'EGGGGEE',
 'EEGGGG',
 'EEEEEE',
 'GAAAAA',
 'EGGGGGG',
 'EEEEEE',
 'EEEEEEE',
 'OOO

In [8]:
def extCRMfromCell_all(input_path="../database/roadmap/css_unit_pickled/", crm_path="../database/remap2022/remap2022_crm_macs2_hg19_v1_0.bed", output_path="../database/remap2022/crm/", limit_len=6):
    files=os.listdir(input_path)
    css_paths=[os.path.join(input_path, file) for file in files if "E" in file]  # list of paths for css of cells
    for css_sample_path in css_paths:
        cut_lst_all=extCRMfromCell(css_sample_path=css_sample_path,crm_path=crm_path,limit_len=limit_len)
         
        file_name=re.search(r'E\d{3}_unitcss_', css_sample_path).group(0)
        output_file=os.path.join(output_path,file_name+"limit_len"+str(limit_len)+".pkl")
        with open(output_file,"wb") as f:
            pickle.dump(cut_lst_all,f)
    return print("All files are saved at {}, with limit_len={}".format(output_path, limit_len))


In [294]:
input_path="../database/roadmap/css_unit_pickled/"
crm_path="../database/remap2022/remap2022_crm_macs2_hg19_v1_0.bed"
output_path_raw="../database/remap2022/crm/"
for num in [6,7,8,9,10]:
    output_path=os.path.join(output_path_raw,"lim"+str(num))
    extCRMfromCell_all(input_path=input_path,crm_path=crm_path,output_path=output_path,limit_len=num)
    

97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are 

In [9]:
def saveCRMforPREall(input_path="../database/remap2022/crm/",output_path="../database/pretrain/crm/",limit_len=6, k=4, crm_not=False): 
    files=os.listdir(os.path.join(input_path,"lim"+str(limit_len)))
    css_all=[]
    for file in files:
        with open(os.path.join(input_path,"lim"+str(limit_len),file), "rb") as f:
            css_lst=pickle.load(f)
        css=flatLst(css_lst)
        _, kmerized_unit_css=css_CUT_Kmer(css, cut_thres=510, k=k)
        css_all.append(kmerized_unit_css)
    css_all_flt=flatLst(css_all)
    if crm_not:
        output_name="not_crm_lim"+str(limit_len)+"_allcell_"+str(k)+"merized.txt"
    else:     
        output_name="crm_lim"+str(limit_len)+"_allcell_"+str(k)+"merized.txt"
    with open(os.path.join(output_path,"lim"+str(limit_len),output_name), "w") as g:
            g.write("\n".join(css_all_flt))
    return print("File is saved at {}".format(output_path))

In [31]:
saveCRMforPREall(input_path="../database/remap2022/crm/",output_path="../database/pretrain/crm/",limit_len=6, k=4)

File is saved at ../database/pretrain/crm/


In [32]:
for limit_len in [6,7,8,9,10]:
    for k in [4,5,6]:
        saveCRMforPREall(input_path="../database/remap2022/crm/",output_path="../database/pretrain/crm/",limit_len=limit_len, k=k)

File is saved at ../database/pretrain/crm/
File is saved at ../database/pretrain/crm/
File is saved at ../database/pretrain/crm/
File is saved at ../database/pretrain/crm/
File is saved at ../database/pretrain/crm/
File is saved at ../database/pretrain/crm/
File is saved at ../database/pretrain/crm/
File is saved at ../database/pretrain/crm/
File is saved at ../database/pretrain/crm/
File is saved at ../database/pretrain/crm/
File is saved at ../database/pretrain/crm/
File is saved at ../database/pretrain/crm/
File is saved at ../database/pretrain/crm/
File is saved at ../database/pretrain/crm/
File is saved at ../database/pretrain/crm/


In [10]:
### cut the css which are NOT crm
def extNOTCRMfromCell(css_sample_path="../database/roadmap/css_unit_pickled/E003_unitcss_woChrM.pkl",crm_path="../database/remap2022/remap2022_crm_macs2_hg19_v1_0.bed",limit_len=4):
    #### load unit-css per chromosome of one cell
    with open(css_sample_path, "rb") as s:
        unit_css=pickle.load(s)
    #### make CRM as a dataframe
    crm_df_fin=crm_df_maker(crm_path=crm_path,limit_len=limit_len)
    
    cut_lst_all=[]
    for chr in range(len(unit_css)):
        unit_css_chr=unit_css[chr]
        chromosome_order = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10',
                            'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19',
                            'chr20', 'chr21', 'chr22', 'chrX', 'chrY']
        crm_df_chr=crm_df_fin[crm_df_fin["chromosome"]==chromosome_order[chr]]

        cut_lst=[]

        # Add initial segment before the first start_loc
        if len(crm_df_chr) > 0:
            first_start = crm_df_chr["start"].iloc[0]
            if first_start > 0:  # Ensure there's a segment before the first start
                cut_lst.append(unit_css_chr[0:first_start])

        # Loop through each entry and add the segment before its start_loc
        for i in range(len(crm_df_chr)):
            if i > 0:  # Skip this for the first entry
                prev_end_loc = crm_df_chr["end"].iloc[i-1]
                start_loc = crm_df_chr["start"].iloc[i]
                # Add the segment from the previous end_loc+1 to the current start_loc-1
                if prev_end_loc + 1 < start_loc:  # Ensure there's a segment in between
                    cut_lst.append(unit_css_chr[prev_end_loc+1:start_loc])

        # Add final segment after the last end_loc
        if len(crm_df_chr) > 0:
            last_end = crm_df_chr["end"].iloc[-1]
            if last_end < len(unit_css_chr) - 1:  # Ensure there's a segment after the last end
                cut_lst.append(unit_css_chr[last_end+1:])

        # for i in range(len(crm_df_chr)):
        #     start_loc=crm_df_chr["start"].iloc[i]
        #     end_loc=crm_df_chr["end"].iloc[i]
        #     # length=crm_df_fin["length"].iloc[i]     
        #     cut=unit_css_chr[start_loc:end_loc+1]
        #     cut_lst.append(cut)
        cut_lst_all.append(cut_lst)
    return cut_lst_all

In [43]:
not_cut_lst_all=extNOTCRMfromCell(css_sample_path="../database/roadmap/css_unit_pickled/E003_unitcss_woChrM.pkl",crm_path="../database/remap2022/remap2022_crm_macs2_hg19_v1_0.bed",limit_len=6)

97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %


In [11]:
def extNotCRMfromCell_all(input_path="../database/roadmap/css_unit_pickled/", crm_path="../database/remap2022/remap2022_crm_macs2_hg19_v1_0.bed", output_path="../database/remap2022/not_crm/", limit_len=6):
    files=os.listdir(input_path)
    css_paths=[os.path.join(input_path, file) for file in files if "E" in file]  # list of paths for css of cells
    for css_sample_path in css_paths:
        cut_lst_all=extNOTCRMfromCell(css_sample_path=css_sample_path,crm_path=crm_path,limit_len=limit_len)
         
        file_name=re.search(r'E\d{3}_unitcss_', css_sample_path).group(0)
        output_file=os.path.join(output_path,file_name+"limit_len"+str(limit_len)+".pkl")
        with open(output_file,"wb") as f:
            pickle.dump(cut_lst_all,f)
    return print("All files are saved at {}, with limit_len={}".format(output_path, limit_len))

In [48]:
input_path="../database/roadmap/css_unit_pickled/"
crm_path="../database/remap2022/remap2022_crm_macs2_hg19_v1_0.bed"
output_path_raw="../database/remap2022/not_crm/"
for num in [6,7,8,9,10]:
    output_path=os.path.join(output_path_raw,"lim"+str(num))
    extNotCRMfromCell_all(input_path=input_path,crm_path=crm_path,output_path=output_path,limit_len=num)

97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are longer than 200x6, which is approx. 0.028 %
97326 out of total 3448721 CRM entries are 

In [12]:
for limit_len in [6,7,8,9,10]:
    for k in [4,5,6]:
        saveCRMforPREall(input_path="../database/remap2022/not_crm/",output_path="../database/pretrain/not_crm/",limit_len=limit_len, k=k, crm_not=True)

File is saved at ../database/pretrain/not_crm/
File is saved at ../database/pretrain/not_crm/
File is saved at ../database/pretrain/not_crm/


In [24]:
def FT_save_CRMnotCRM(input_path1="../database/pretrain/crm/",input_path2="../database/pretrain/not_crm/",
                 output_path="../database/fine_tune/crm/",
                 len_tr=20000, len_dev=1000,
                 limit_len=10, k=4, rand_state=1):
    
    crm_path=os.path.join(input_path1,"lim"+str(limit_len))
    tar_cl1=[file for file in os.listdir(crm_path) if str(k) in file][0]
    cl1_path=os.path.join(crm_path,tar_cl1)
    # print(cl1_path)

    not_crm_path=os.path.join(input_path2,"lim"+str(limit_len))
    tar_cl2=[file for file in os.listdir(not_crm_path) if str(k) in file][0]
    cl2_path=os.path.join(not_crm_path,tar_cl2)
    # print(cl2_path)

    with open(cl1_path,"r") as f1:
        cl1_lst=[line.strip() for line in f1]
    with open(cl2_path, "r") as f2:
        cl2_lst=[line.strip() for line in f2]
   
    # make it dataframe
    df_cl1=pd.DataFrame(cl1_lst, columns=["sequence"])  #  crm
    df_cl1["label"]=1
    df_cl2=pd.DataFrame(cl2_lst, columns=["sequence"])  #  not_crm
    df_cl2["label"]=0

    # make them have the same length
    # if len(df_cl1)>len(df_cl2):
    #     df_cl1=df_cl1.sample(n=len(df_cl2), random_state=1) # use the same random state for reproducibility
    # elif len(df_cl1)<len(df_cl2):
    #     df_cl2=df_cl2.sample(n=len(df_cl1), random_state=1) # use the same random state for reproducibility
    # assert len(df_cl1)==len(df_cl2), "Check the data length."
    if len(df_cl1)>len(df_cl2):
        df_cl1=df_cl1.sample(n=len(df_cl2), random_state=rand_state) # use the same random state for reproducibility
    elif len(df_cl1)<len(df_cl2):
        df_cl2=df_cl2.sample(n=len(df_cl1), random_state=rand_state) # use the same random state for reproducibility
    assert len(df_cl1)==len(df_cl2), "Check the data length."

    df_all=pd.concat([df_cl1,df_cl2]).sample(frac=1).reset_index(drop=True) 

    # cutting into train and dev
    assert len(df_all)> len_tr+len_dev, "Not enough data length."
    df_train=df_all[:len_tr]
    df_dev=df_all[len_tr:len_tr+len_dev]    


    output_path_fin=os.path.join(output_path,str(k)+"mer","lim"+str(limit_len),"crm_not_crm"+"_"+str(rand_state))

    # Check if the directory exists
    if not os.path.exists(output_path_fin):
        # If the directory does not exist, create it
        os.makedirs(output_path_fin)
    
    train_name=os.path.join(output_path_fin,"train.tsv")
    dev_name=os.path.join(output_path_fin,"dev.tsv")

    df_train.to_csv(train_name, sep="\t", index=False)
    df_dev.to_csv(dev_name, sep="\t", index=False)
    print("Fine tuning files are saved at ", output_path_fin)

    return df_train,df_dev

In [25]:
for ran_state in [1,2,3,4,5,6,7,8,9]:
    for lim_len in [6,7,8,9,10]:
        FT_save_CRMnotCRM(input_path1="../database/pretrain/crm/",input_path2="../database/pretrain/not_crm/",
                        output_path="../database/fine_tune/crm/",len_tr=20000, len_dev=1000,
                        limit_len=lim_len,k=4,rand_state=ran_state)

Fine tuning files are saved at  ../database/fine_tune/crm/4mer/lim6/crm_not_crm_1
Fine tuning files are saved at  ../database/fine_tune/crm/4mer/lim7/crm_not_crm_1
Fine tuning files are saved at  ../database/fine_tune/crm/4mer/lim8/crm_not_crm_1
Fine tuning files are saved at  ../database/fine_tune/crm/4mer/lim9/crm_not_crm_1
Fine tuning files are saved at  ../database/fine_tune/crm/4mer/lim10/crm_not_crm_1
Fine tuning files are saved at  ../database/fine_tune/crm/4mer/lim6/crm_not_crm_2
Fine tuning files are saved at  ../database/fine_tune/crm/4mer/lim7/crm_not_crm_2
Fine tuning files are saved at  ../database/fine_tune/crm/4mer/lim8/crm_not_crm_2
Fine tuning files are saved at  ../database/fine_tune/crm/4mer/lim9/crm_not_crm_2
Fine tuning files are saved at  ../database/fine_tune/crm/4mer/lim10/crm_not_crm_2
Fine tuning files are saved at  ../database/fine_tune/crm/4mer/lim6/crm_not_crm_3
Fine tuning files are saved at  ../database/fine_tune/crm/4mer/lim7/crm_not_crm_3
Fine tuning fi